In [1]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Input
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional,MaxPooling1D
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from keras.optimizers import Adam

In [2]:
# Load data
df_train=pd.read_csv('../input/bt5153-train-test-bert-full-sentence/train_data_bert_fullsent.csv',dtype={'label':str})
df_test=pd.read_csv('../input/bt5153-train-test-bert-full-sentence/test_data_bert_fullsent.csv',dtype={'label':str})

In [3]:
df_train.head()

,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label
0,9161,"Delivery took more than a week, short expiry d...",2,1,1,0,"[-0.06329992, -0.32380387, 0.35608995, -0.1268...","[0.93797976, 0.25249124, 1.1054325, 0.49609792...","[-0.19600664, -0.4481723, 0.20494524, -0.39789...","[-0.046782605, -0.27473888, 0.2780265, 0.03694...","[0.4727717, 0.23103717, 0.7197178, 0.6083572, ...","[-0.022450736, -0.34166092, 0.13424423, -0.170...",110
1,8198,"Quality is so-so, loops are too huge resulting...",3,0,1,0,"[0.19020107, -0.3453789, 0.3494842, 0.14938574...","[1.1594802, 0.60506886, 1.084206, 0.8051733, 0...","[0.04546422, -0.29597518, -0.35977763, -0.4039...","[0.1399654, -0.11672436, 0.23911656, 0.1187961...","[0.8445691, 0.4961368, 0.7105234, 0.6109599, 0...","[0.12642525, -0.1857245, 0.08149249, -0.019622...",010
2,5314,Received within 3days. Well packed in a carton...,4,0,1,0,"[-0.028732965, -0.14712504, 0.36105582, 0.0254...","[1.3182856, 0.7953221, 1.3915914, 0.87499446, ...","[-0.38282838, -0.22101544, 0.6115111, -0.33658...","[0.07374873, -0.01620108, 0.20269494, 0.149083...","[0.76418287, 0.48008785, 0.97139245, 0.5615782...","[0.026645288, -0.27306008, 0.20750262, 0.05986...",010
3,17589,Not sure if these were the authentic as I brou...,3,0,1,0,"[0.3033499, 0.02644875, 0.16410081, 0.04581299...","[1.2807441, 0.85781705, 0.80655, 0.49130562, 0...","[-0.078149214, 0.117149524, 0.04559862, -0.218...","[0.18770164, 0.114881225, 0.08257794, 0.121869...","[0.9882787, 0.5043827, 0.54232115, 0.44097498,...","[0.23179623, 0.050810635, 0.07551272, -0.15587...",010
4,31446,its a fashion mask. no filter so no protection...,3,0,1,0,"[-0.18616518, -0.38300017, 0.52194715, 0.18413...","[0.65815246, 0.20897251, 1.0151551, 0.48899576...","[-0.29987, -0.26352054, 0.3190687, -0.22115439...","[-0.0903255, -0.1808794, 0.33290026, 0.2622123...","[0.66676205, 0.099603325, 0.8990194, 0.5398592...","[-0.34698877, -0.3470553, 0.3122651, 0.0037901...",010


In [4]:
f=open('../input/5153-bert-feature-extraction-negative/negative_bert_layer0_dic.txt','r')
neg_bert_layer0=(f.read())
neg_bert_layer0=eval(neg_bert_layer0)
df_neg_feature=pd.DataFrame({'review_id':neg_bert_layer0.keys(),
                            'neg_distil_bert_layer0':neg_bert_layer0.values()})
df_train=df_train.merge(df_neg_feature,on='review_id',how='left')
df_test=df_test.merge(df_neg_feature,on='review_id',how='left')

In [5]:
f=open('../input/5153-bert-feature-extraction-negative/negative_distil_bert_avg_dic.txt','r')
neg_bert_avg=(f.read())
neg_bert_avg=eval(neg_bert_avg)
df_neg_feature=pd.DataFrame({'review_id':neg_bert_avg.keys(),
                            'neg_distil_bert_avg':neg_bert_avg.values()})

df_train=df_train.merge(df_neg_feature,on='review_id',how='left')
df_test=df_test.merge(df_neg_feature,on='review_id',how='left')

In [6]:
def BiLstm_model(X_train, y_train):
    model = Sequential()

    model.add(Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.1))) 

    model.add(Dense(512, activation='relu'))  

    model.add(Dense(128, activation='relu')) 

    model.add(Dense(3, activation='sigmoid')) 

    optimizer=Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    model.compile(loss='BinaryCrossentropy', optimizer=optimizer, metrics=['accuracy'])

    epochs = 100
    batch_size = 32

    history = model.fit(X_train, y_train, 
            epochs=epochs, 
            batch_size=batch_size,
            validation_split=0.1,
            callbacks=[EarlyStopping(monitor='val_loss', 
            patience=3, min_delta=0.0001)])
    return model

In [7]:
def get_results(model,X_test, y_test):


    results_prob=model.predict(X_test)
    results_pred=np.copy(results_prob)
    results_pred[results_prob>=0.5]=1
    results_pred[results_prob<0.5]=0
    results_pred=results_pred.astype('int')

    print('roc_auc_score_delivery:',round(roc_auc_score(list(df_test['delivery']), results_prob[:,0]),3))
    print('roc_auc_score_product:',round(roc_auc_score(list(df_test['product']), results_prob[:,1]),3))
    print('roc_auc_score_service:',round(roc_auc_score(list(df_test['service']), results_prob[:,2]),3))
    print('macro_fl_delivery:',round(f1_score(list(df_test['delivery']), results_pred[:,0],average='macro'),3))
    print('macro_fl_product:',round(f1_score(list(df_test['product']), results_pred[:,1],average='macro'),3))
    print('macro_fl_service:',round(f1_score(list(df_test['service']), results_pred[:,2],average='macro'),3))


In [8]:
def data_process_1D(feature_name):
    X_train=np.array([eval(str(x)) for x in list(df_train[feature_name])])
    X_train=X_train[:,np.newaxis,:]
    y_train=np.array(df_train[['delivery','product','service']])

    X_test=np.array([eval(str(x)) for x in list(df_test[feature_name])])
    X_test=X_test[:,np.newaxis,:]
    y_test=np.array(df_test[['delivery','product','service']])
    
    return X_train,y_train,X_test,y_test

### use distill_bert_layer0

In [9]:
X_train,y_train,X_test,y_test=data_process_1D('distil_bert_layer0')

model_1=BiLstm_model(X_train,y_train)
get_results(model_1,X_test,y_test)

Epoch 1/100
30/30 [==============================] - 7s 60ms/step - loss: 0.6284 - accuracy: 0.5842 - val_loss: 0.5471 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 1s 27ms/step - loss: 0.5133 - accuracy: 0.6584 - val_loss: 0.5260 - val_accuracy: 0.5981
Epoch 3/100
30/30 [==============================] - 1s 27ms/step - loss: 0.5036 - accuracy: 0.6256 - val_loss: 0.4799 - val_accuracy: 0.6075
Epoch 4/100
30/30 [==============================] - 1s 27ms/step - loss: 0.4340 - accuracy: 0.6694 - val_loss: 0.4384 - val_accuracy: 0.6916
Epoch 5/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3810 - accuracy: 0.7429 - val_loss: 0.4149 - val_accuracy: 0.7477
Epoch 6/100
30/30 [==============================] - 1s 26ms/step - loss: 0.3403 - accuracy: 0.7851 - val_loss: 0.4032 - val_accuracy: 0.7383
Epoch 7/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3326 - accuracy: 0.7759 - val_loss: 0.4002 - val_accuracy: 0.7477
Epoch 

### use distill_bert_avg 

In [10]:
X_train,y_train,X_test,y_test=data_process_1D('distil_bert_avg')

model_2=BiLstm_model(X_train,y_train)
get_results(model_2,X_test,y_test)


Epoch 1/100
30/30 [==============================] - 7s 47ms/step - loss: 0.6406 - accuracy: 0.6377 - val_loss: 0.5472 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 1s 26ms/step - loss: 0.5255 - accuracy: 0.6463 - val_loss: 0.5084 - val_accuracy: 0.5981
Epoch 3/100
30/30 [==============================] - 1s 26ms/step - loss: 0.4705 - accuracy: 0.6446 - val_loss: 0.4543 - val_accuracy: 0.6729
Epoch 4/100
30/30 [==============================] - 1s 26ms/step - loss: 0.3972 - accuracy: 0.7300 - val_loss: 0.4154 - val_accuracy: 0.7009
Epoch 5/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3466 - accuracy: 0.7705 - val_loss: 0.3835 - val_accuracy: 0.7570
Epoch 6/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3183 - accuracy: 0.8182 - val_loss: 0.3948 - val_accuracy: 0.7290
Epoch 7/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3060 - accuracy: 0.7865 - val_loss: 0.3648 - val_accuracy: 0.7757
Epoch 

### use neg_distil_bert_layer0

In [11]:
X_train,y_train,X_test,y_test=data_process_1D('neg_distil_bert_layer0')

model_3=BiLstm_model(X_train,y_train)
get_results(model_3,X_test,y_test)

Epoch 1/100
30/30 [==============================] - 7s 46ms/step - loss: 0.6417 - accuracy: 0.4913 - val_loss: 0.5511 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 1s 26ms/step - loss: 0.5284 - accuracy: 0.6332 - val_loss: 0.5164 - val_accuracy: 0.5981
Epoch 3/100
30/30 [==============================] - 1s 27ms/step - loss: 0.4873 - accuracy: 0.6425 - val_loss: 0.4535 - val_accuracy: 0.6636
Epoch 4/100
30/30 [==============================] - 1s 26ms/step - loss: 0.4209 - accuracy: 0.7287 - val_loss: 0.3850 - val_accuracy: 0.7383
Epoch 5/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3606 - accuracy: 0.7807 - val_loss: 0.3497 - val_accuracy: 0.7850
Epoch 6/100
30/30 [==============================] - 1s 28ms/step - loss: 0.3482 - accuracy: 0.7718 - val_loss: 0.3362 - val_accuracy: 0.7664
Epoch 7/100
30/30 [==============================] - 1s 27ms/step - loss: 0.3070 - accuracy: 0.8171 - val_loss: 0.3256 - val_accuracy: 0.7757
Epoch 

## use_neg_distllbert_avg

In [12]:
X_train,y_train,X_test,y_test=data_process_1D('neg_distil_bert_avg')

model_4=BiLstm_model(X_train,y_train)
get_results(model_4,X_test,y_test)

Epoch 1/100
30/30 [==============================] - 7s 46ms/step - loss: 0.6456 - accuracy: 0.5772 - val_loss: 0.5540 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 1s 27ms/step - loss: 0.5475 - accuracy: 0.6140 - val_loss: 0.5223 - val_accuracy: 0.5981
Epoch 3/100
30/30 [==============================] - 1s 27ms/step - loss: 0.5022 - accuracy: 0.6154 - val_loss: 0.4661 - val_accuracy: 0.6075
Epoch 4/100
30/30 [==============================] - 1s 27ms/step - loss: 0.4565 - accuracy: 0.6786 - val_loss: 0.4062 - val_accuracy: 0.7477
Epoch 5/100
30/30 [==============================] - 1s 35ms/step - loss: 0.3608 - accuracy: 0.7958 - val_loss: 0.3735 - val_accuracy: 0.7477
Epoch 6/100
30/30 [==============================] - 1s 28ms/step - loss: 0.3312 - accuracy: 0.7981 - val_loss: 0.3532 - val_accuracy: 0.7664
Epoch 7/100
30/30 [==============================] - 1s 27ms/step - loss: 0.2942 - accuracy: 0.8286 - val_loss: 0.3331 - val_accuracy: 0.7757
Epoch 

# 2d FEATURE


In [13]:
twoD_feature=np.load("../input/5153-bert-feature-extraction-negative/distil_bert_dic_2D.pkl",allow_pickle=True)
df_neg_feature=pd.DataFrame({'review_id':twoD_feature.keys(),
                            'neg_2d_distill':twoD_feature.values()})
df_train=df_train.merge(df_neg_feature,on='review_id',how='left')
df_test=df_test.merge(df_neg_feature,on='review_id',how='left')

In [14]:
twoD_feature=np.load("../input/5153-bert-fullsentence-featureextraction-final/distil_full_bert_dic_2D.pkl",allow_pickle=True)
df_feature=pd.DataFrame({'review_id':twoD_feature.keys(),
                         '2d_distill':twoD_feature.values()})
df_train=df_train.merge(df_feature,on='review_id',how='left')
df_test=df_test.merge(df_feature,on='review_id',how='left')

In [15]:
df_train.columns

Index(['review_id', 'review_content', 'review_stars', 'delivery', 'product',
       'service', 'bert_avg', 'bert_max', 'bert_layer0', 'distil_bert_avg',
       'distil_bert_max', 'distil_bert_layer0', 'label',
       'neg_distil_bert_layer0', 'neg_distil_bert_avg', 'neg_2d_distill',
       '2d_distill'],
      dtype='object')

In [16]:
def data_process_2D(feature_name):
    X_train=np.array([x for x in list(df_train[feature_name])])
    X_train=X_train[:,0,:,:]
    y_train=np.array(df_train[['delivery','product','service']])

    X_test=np.array([x for x in list(df_test[feature_name])])
    X_test=X_test[:,0,:,:]
    y_test=np.array(df_test[['delivery','product','service']])
    
    return X_train,y_train,X_test,y_test

In [17]:
X_train,y_train,X_test,y_test=data_process_2D('neg_2d_distill')

model_5=BiLstm_model(X_train,y_train)
get_results(model_5,X_test,y_test)

Epoch 1/100
30/30 [==============================] - 58s 2s/step - loss: 0.6070 - accuracy: 0.5684 - val_loss: 0.5094 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 50s 2s/step - loss: 0.4554 - accuracy: 0.6709 - val_loss: 0.3858 - val_accuracy: 0.7570
Epoch 3/100
30/30 [==============================] - 49s 2s/step - loss: 0.3450 - accuracy: 0.7908 - val_loss: 0.3709 - val_accuracy: 0.7570
Epoch 4/100
30/30 [==============================] - 50s 2s/step - loss: 0.3224 - accuracy: 0.8237 - val_loss: 0.3406 - val_accuracy: 0.7664
Epoch 5/100
30/30 [==============================] - 49s 2s/step - loss: 0.2571 - accuracy: 0.8523 - val_loss: 0.3275 - val_accuracy: 0.8224
Epoch 6/100
30/30 [==============================] - 49s 2s/step - loss: 0.2192 - accuracy: 0.8680 - val_loss: 0.3183 - val_accuracy: 0.8411
Epoch 7/100
30/30 [==============================] - 50s 2s/step - loss: 0.1887 - accuracy: 0.9001 - val_loss: 0.3395 - val_accuracy: 0.7944
Epoch 8/100
3

In [18]:
X_train,y_train,X_test,y_test=data_process_2D('2d_distill')

model_5=BiLstm_model(X_train,y_train)
get_results(model_5,X_test,y_test)

Epoch 1/100
30/30 [==============================] - 46s 1s/step - loss: 0.5920 - accuracy: 0.5636 - val_loss: 0.4816 - val_accuracy: 0.5981
Epoch 2/100
30/30 [==============================] - 39s 1s/step - loss: 0.4149 - accuracy: 0.7189 - val_loss: 0.3699 - val_accuracy: 0.7664
Epoch 3/100
30/30 [==============================] - 40s 1s/step - loss: 0.3224 - accuracy: 0.8244 - val_loss: 0.3889 - val_accuracy: 0.7196
Epoch 4/100
30/30 [==============================] - 39s 1s/step - loss: 0.2922 - accuracy: 0.8019 - val_loss: 0.3451 - val_accuracy: 0.7850
Epoch 5/100
30/30 [==============================] - 40s 1s/step - loss: 0.2288 - accuracy: 0.8574 - val_loss: 0.3339 - val_accuracy: 0.8037
Epoch 6/100
30/30 [==============================] - 39s 1s/step - loss: 0.1939 - accuracy: 0.8802 - val_loss: 0.3340 - val_accuracy: 0.8131
Epoch 7/100
30/30 [==============================] - 40s 1s/step - loss: 0.1937 - accuracy: 0.8997 - val_loss: 0.3547 - val_accuracy: 0.8131
Epoch 8/100
3